In [ ]:
import http.client

conn = http.client.HTTPSConnection("free-api-live-football-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "3c99262600msh5da2d3d13033579p1c677ejsnc94020f9ff0e",

    'x-rapidapi-host': "free-api-live-football-data.p.rapidapi.com"
}

conn.request("GET", "/football-get-trendingnews", headers=headers)
conn.request("GET", "/football-get-league-news?leagueid=47&page=1", headers=headers) # league
conn.request("GET", "/football-get-team-news?teamid=8650&page=1", headers=headers) # teams
conn.request("GET", "/football-get-player-news?playerid=8650&page=1", headers=headers) # player

conn.request("GET", "/football-get-all-transfers?page=1", headers=headers)
conn.request("GET", "/football-get-top-transfers?page=1", headers=headers)
conn.request("GET", "/football-get-market-value-transfers?page=1", headers=headers)
conn.request("GET", "/football-get-league-transfers?leagueid=47", headers=headers)
conn.request("GET", "/football-get-team-contract-extension-transfers?teamid=8650", headers=headers)
conn.request("GET", "/football-get-team-players-in-transfers?teamid=8650", headers=headers)
conn.request("GET", "/football-get-team-players-out-transfers?teamid=8650", headers=headers)


res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))